In [ ]:
import numpy as np
from numpy import array as arr
from numpy import zeros as zarr

import pandas as pd
from pandas import DataFrame as df
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
mpl.rcParams["font.serif"] = "CMU serif"
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.rm"] = "CMU serif"
mpl.rcParams["mathtext.it"] = "CMU serif:italic"
mpl.rcParams["mathtext.bf"] = "CMU serif:bold"
mpl.rcParams["font.family"] = "serif"

# import pylandau
# from scipy import curve_fit
import scipy
import uncertainties as unc
from uncertainties import unumpy as unp
from uncertainties.unumpy import uarray as uarr
from uncertainties.unumpy import nominal_values as val
from uncertainties.unumpy import std_devs as dev
from uncertainties import ufloat as uf
# import ROOT

# import my plotting stuff
import sys
sys.path.append('./PythonHelpers/')
from PlotLib import Plotting
import PlotLib.Histogramming as Hist
import PlotLib.Plotting as Plot
from CSVimporter.runSettings import load_settings
from CSVimporter.importer import load_run
from CSVimporter.importer import dict_to_arr

import ROOT



def df_newcol(dframe, column, value, loc=-1):
    if column in dframe.columns and type(dframe.loc[0, column]) == type(value):
        print("WARNING: column", column, " already exists and has the same type")
        return
    if loc == -1:
        loc = len(dframe.columns)
    elif column in dframe.columns:
        loc = len(dframe.columns)
    try:
        try:
            dframe.insert(loc, column, pd.Series([value.copy()]*dframe.size, dtype=object))
        except:
            dframe.insert(loc, column, pd.Series([value.Clone()]*dframe.size, dtype=object))
    except:
        dframe.insert(loc, column, pd.Series([value]*dframe.size, dtype=object))
        
def df_query(frame, args, col, single=True):
    if type(args) == dict:
        query_str = " & ".join(["{:s}=={:n}".format(key, value) for key, value in args.items()])
    elif type(args) == list:
        query_str = " & ".join(["{:s}=={:n}".format(key, value) for key, value in args])
    elif type(args) == str:
        query_str = args
    index = frame.query(query_str).index
    if single:
        if len(index) != 1:
            raise Exception("Query returned {:n} entries".format(len(index)))
        index = index[0]
    return frame.at[index, col]
        
def get_hist_TH1(hHist):
    binN = hHist.GetNbinsX()
    bins = zarr(binN+1)    
    vals = zarr(binN)
    bins[0] = hHist.GetBinLowEdge(1)
    for i in range(binN):
        bins[i+1] = hHist.GetBinLowEdge(i+2)
        vals[i] = hHist.GetBinContent(i+1)
    return vals, bins

In [ ]:
thr = 220
runID = 190
SCA_trk = 30
SCA_assoc = 100
SEC = 0.5
MTPs = arr([3,4,5,6])
MTP = 6

FilePath = "/home/jona/DESY/analysis_TB/output/scan_tracking/"
FileName = FilePath+str(runID) + "_thr{:n}".format(thr) + "_TrkCut{:n}".format(SCA_trk) + "_TrkPlanes{:n}".format(MTP) + "_AssocCut{:n}".format(SCA_assoc) + "_SEC{:.1f}".format(SEC) + ".root"

# output/scan_tracking/190_thr220_TrkCut150_TrkPlanes6_AssocCut60_SEC0.5.root

rFile = ROOT.TFile(FileName)
if not rFile:
    print("File not found")
    rFile.Close()

# Get the efficiency
rDir = rFile.Get("AnalysisEfficiency/DSO9254A_0")
if not rDir:
    print("thr: {:n}, trk_Cut: {:n}, trk_planes: {:n}, SCA_assoc: {:n}".format(thr, SCA_trk, MTP, SCA_assoc))
    raise Exception("Dir. AnalysisEfficiency/DSO9254A_0 not found in root file")
else:
    TEff = rDir.Get("eTotalEfficiency")    
    if not TEff:
        raise Exception("eTotalEfficiency not found in root file")
    else:
        eff = arr([TEff.GetEfficiency(1), TEff.GetEfficiencyErrorLow(1), TEff.GetEfficiencyErrorUp(1)])
        
# Get the fake rate
rDir = rFile.Get("AnalysisEfficiency/DSO9254A_0/fake_rate")
if not rDir:
    raise Exception("Dir. AnalysisEfficiency/DSO9254A_0/fake_rate not found in root file")
else:
    hFakePixelPerEvt = rDir.Get("hFakePixelPerEvent")
    if not hFakePixelPerEvt:
        raise Exception("hFakePixelPerEvent not found in root file")
    else:
        fakeRate = arr([hFakePixelPerEvt.GetMean(), hFakePixelPerEvt.GetRMS()])

# Get the number of tracks per event histogram
rDir = rFile.Get("Tracking4D")
if not rDir:
    raise Exception("Dir. Tracking4D not found in root file")
else:
    hTrackMult = rDir.Get("tracksPerEvent")
    if not hTrackMult:
        raise Exception("tracksPerEvent not found in root file")
    else:
        TrackMult_vals, TrackMult_bins = get_hist_TH1(hTrackMult)
        TrackMult = uf(hTrackMult.GetMean(), hTrackMult.GetRMS())
        
# Get the tracking chi2red histogram
rDir = rFile.Get("Tracking4D")
if not rDir:
    raise Exception("Dir. Tracking4D not found in root file")
else:
    hChi2red = rDir.Get("trackChi2ndof")
    if not hChi2red:
        raise Exception("trackChi2ndof not found in root file")
    else:
        Chi2red_vals, Chi2red_bins = get_hist_TH1(hChi2red)
        
# Get the number of tracks with an associated DUT hit
nameDir = "AnalysisTracksDUT/adenium_3"
nameObj = "track_assocDUT_intersect_global"
rDir = rFile.Get(nameDir)
if not rDir:
    raise Exception("Dir.", nameDir, "not found in root file")
else:
    hTrackAssocDUTIntersectGlobal = rDir.Get(nameObj)
    if not hChi2red:
        raise Exception("Object", nameObj, "not found in root file at dir.", nameDir)
    else:
        AssocTracks = hTrackAssocDUTIntersectGlobal.GetEntries()



# Get the association residual histogram
nameDir = f"AnalysisDUT/DSO9254A_0/local_residuals"
rDir = rFile.Get(nameDir)
if not rDir:
    print(FileName)
    raise Exception("Dir.", nameDir, "not found in root file")
else:
    # Get residual-x histogram & fit to get gaussian
    nameObj = "residualsX"
    hResidualX = rDir.Get(nameObj)
    if not hResidualX:
        raise Exception("Object", nameObj, "not found in root file at dir.", nameDir)
    else:
        valsX, binsX = get_hist_TH1(hResidualX)
        print("RMS residualX:", hResidualX.GetRMS())
        Range = hResidualX.GetRMS()*2
        rangeBin0 = hResidualX.FindBin(-Range)
        rangeBin1 = hResidualX.FindBin(Range)
        hResidualX.GetXaxis().SetRange(rangeBin0, rangeBin1)
        print("RMS(95%) residualX:", hResidualX.GetRMS()*5)
        
    # Get residual-y histogram & fit to get gaussian
    nameObj = "residualsY"
    hResidualY = rDir.Get(nameObj)
    if not hResidualY:
        raise Exception("Object", nameObj, "not found in root file at dir.", nameDir)
    else:
        valsY, binsY = get_hist_TH1(hResidualY)
        print("RMS residualY:", hResidualY.GetRMS())
        Range = hResidualY.GetRMS()*2
        rangeBin0 = hResidualY.FindBin(-Range)
        rangeBin1 = hResidualY.FindBin(Range)
        hResidualY.GetXaxis().SetRange(rangeBin0, rangeBin1)
        print("RMS(95%) residualY:", hResidualY.GetRMS()*5)
        
        
rFile.Close()

runSettings = load_settings(runID)


In [ ]:
valsX.sort()
valsY.sort()

i_trunc = int(len(valsX)*0.02)
valsXtrunc = valsX[i_trunc:-i_trunc]
valsYtrunc = valsY[i_trunc:-i_trunc]

print("X")
print("RMS: ", valsX.std())
print("RMS(96%):", valsXtrunc.std())

print("Y")
print("RMS: ", valsY.std())
print("RMS(96%):", valsYtrunc.std())

In [ ]:
# draw residual sigma histogram
fig, ax = Hist.create_fig(1,1, figsize=(4,3))

# x
x = arr(range(N_Planes))
y = residual_fitp[:,0,1]
yerr = arr([dev(residual_fitperr[i,0,1]) for i in range(N_Planes)]) 
ax.errorbar(x, y*1000, yerr=yerr*1000, fmt=".", label="x residual", color="black")

y = residual_fitp[:,1,1]
yerr = arr([dev(residual_fitperr[i,1,1]) for i in range(N_Planes)])
ax.errorbar(x, y*1000, yerr=yerr*1000, fmt=".", label="y residual", color="darkred")


ax.set_xticks(range(N_Planes))
ax.set_xlim(-0.5, N_Planes-0.5)

Hist.finalize(runSettings, fig, ax, title="Tracking residuals\nGaussian width\n(from fit)", xlabel=r"adenium plane (along beam)", ylabel=r"residual Gaussian width / um", param_narrow=True, grid=False, legend_loc="lower right", thr=thr, ER1=False)

print("cuts at 3 sigma of largest residuals")
print("x:", max(residual_fitp[:,0,1])*5*1000)
print("y:", max(residual_fitp[:,1,1])*5*1000)

In [ ]:
fig, ax = Hist.create_fig(1,1, figsize=(8,6))


for plane in range(6):
    ax.stairs(residual_vals[plane,0]/np.sum(residual_vals[plane,0]), residual_bins[plane,0]*1000, label=f"adenium plane {plane}", color=f"C{plane}")
    binWidth = residual_bins[plane,0][1]-residual_bins[plane,0][0]

ax.set_xlim(-15,15)
Hist.finalize(runSettings, fig, ax, title=f"Tracking residuals", xlabel=r"track residual / um", ylabel=f"relative frequency / per {binWidth*1000:.2} um", param_narrow=True, grid=False, ER1=False)